In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

### GPU 확인

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


### NLLB 모델 로드 (3.3B 버전 - 더 좋은 품질)

In [3]:
model_name = "facebook/nllb-200-3.3B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_name, 
    torch_dtype=torch.float16,  # 메모리 절약
    device_map="auto"  # GPU 자동 할당
)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
def translate_text(text, target_lang_code):
    """
    영어 텍스트를 목표 언어로 번역
    """
    # 입력 토큰화
    inputs = tokenizer(
        text, 
        return_tensors="pt", 
        padding=True, 
        truncation=True,
        max_length=512
    ).to(device)
    
    # 번역 생성
    translated_tokens = model.generate(
        **inputs,
        forced_bos_token_id=tokenizer.convert_tokens_to_ids(target_lang_code),
        max_length=512,
        num_beams=5,
        early_stopping=True
    )
    
    # 결과 디코딩
    result = tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
    return result

In [6]:
# 주요 30개 언어 코드 (NLLB 기준)
target_languages = {
    'Korean': 'kor_Hang',
    'Japanese': 'jpn_Jpan',
    'Chinese (Simplified)': 'zho_Hans',
    'Chinese (Traditional)': 'zho_Hant',
    'Spanish': 'spa_Latn',
    'French': 'fra_Latn',
    'German': 'deu_Latn',
    'Italian': 'ita_Latn',
    'Portuguese': 'por_Latn',
    'Russian': 'rus_Cyrl',
    'Arabic': 'arb_Arab',
    'Hindi': 'hin_Deva',
    'Bengali': 'ben_Beng',
    'Thai': 'tha_Thai',
    'Vietnamese': 'vie_Latn',
    'Indonesian': 'ind_Latn',
    'Turkish': 'tur_Latn',
    'Dutch': 'nld_Latn',
    'Polish': 'pol_Latn',
    'Swedish': 'swe_Latn',
    'Norwegian': 'nob_Latn',
    'Finnish': 'fin_Latn',
    'Greek': 'ell_Grek',
    'Hebrew': 'heb_Hebr',
    'Persian': 'pes_Arab',
    'Urdu': 'urd_Arab',
    'Tamil': 'tam_Taml',
    'Telugu': 'tel_Telu',
    'Malay': 'zsm_Latn',
    'Swahili': 'swh_Latn'
}

# 번역 실행 예시
input_text = "Traveling allows people to explore new cultures, meet different people, and experience life from a fresh perspective. Whether it's walking through the streets of Paris or hiking in the mountains of Peru, each journey brings unforgettable memories. It also helps individuals grow by stepping out of their comfort zones and embracing the unknown. In a world full of diversity, traveling reminds us how beautifully connected we all are."
results = {}

for lang_name, lang_code in target_languages.items():
    try:
        translated = translate_text(input_text, lang_code)
        results[lang_name] = translated
        print(f"{lang_name}: {translated}")
    except Exception as e:
        print(f"Error translating to {lang_name}: {e}")

Korean: 여행은 사람들이 새로운 문화를 탐구하고, 다른 사람들을 만나고, 새로운 관점에서 삶을 경험할 수 있게 해줍니다. 파리의 거리를 걷거나 페루의 산을 산책하든, 각 여행은 잊을 수 없는 추억을 가져옵니다. 또한 개인이 자신의 편안한 영역을 벗어나서 알려지지 않은 것을 포용함으로써 성장하도록 돕습니다. 다양성으로 가득 찬 세계에서 여행은 우리 모두가 얼마나 아름답게 연결되어 있는지 상기시킵니다.
Japanese: 旅行は,人々が新しい文化を探求し,異なる人々と出会い,新しい視点から人生を経験することを可能にします.パリの街を歩いたり,ペルーの山々をハイキングしたりしても,それぞれの旅は忘れられない思い出をもたらします.また,個人が快適なゾーンから抜け出し,未知のものを抱きしめることで成長するのを助けます.多様性に満ちた世界では,旅行は,私たちがどれほど美しく繋がっているかを思い出させます.
Chinese (Simplified): 旅行使人们能够探索新文化,遇见不同的人,从全新的角度体验生活.无论是穿越巴黎街道还是徒步在秘鲁山脉,每一次旅行都会带来难忘的回忆.它还可以帮助个人通过走出舒适区并拥抱未知的方式成长.在一个充满多样性的世界里,旅行提醒我们,我们都有多么美丽的联系.
Chinese (Traditional): 旅行讓人們能探索新文化,遇見不同的人,從新角度體驗生活. 不論是穿越巴黎街道或徒步在秘魯山脈, 每次旅行都帶來難以忘懷的回憶. 它也幫助個人成長, 走出他們的舒適區域, 擁抱未知的世界. 在一個充滿多樣性的世界裡, 旅行提醒我們我們都是多麼美麗的連結.
Spanish: Viajar permite a las personas explorar nuevas culturas, conocer gente diferente y experimentar la vida desde una nueva perspectiva. Ya sea caminando por las calles de París o caminando en las montañas de Perú, cada viaje trae recuerdos inolvidables. También ayuda a